In [3]:
from alco_analysis.loading_data import load_data
from alco_analysis.correlation import *
from alco_analysis.stats import count_concession, summary, merge_df_on_city, different_correlations

df_conc, df_events, df_cities = load_data("data/concession.csv", 
                                                     "data/events.csv",
                                                     "data/cities.csv")

            city        date          woj
0       warszawa  2025-02-28  MAZOWIECKIE
1        kartuzy  2025-03-03    POMORSKIE
2      ciechanow  2025-02-21  MAZOWIECKIE
3  bielsko-biala  2025-03-30      ŚLĄSKIE
4      bienkowka  2025-04-20  MAŁOPOLSKIE

###CONCESSION DATAFRAME###

            city        date          woj
0       warszawa  2025-02-28  MAZOWIECKIE
1        kartuzy  2025-03-03    POMORSKIE
2      ciechanow  2025-02-21  MAZOWIECKIE
3  bielsko-biala  2025-03-30      ŚLĄSKIE
4      bienkowka  2025-04-20  MAŁOPOLSKIE

###EVENTS DATAFRAME###

   Nr KP/M PSP  RAZEM Pożar (P)  Mały (P/M)  Średni (P/Ś)  Duży (P/D)  \
0       101000             2301        2257            39           5   
1       103000              468         453            14           1   
2       104000              302         295             6           1   
3       105000              303         291            11           1   
4       106000              151         139            10           1   

   

In [ ]:
merged = (df_conc
          .merge(df_cities,
                 on=['city'],
                 how='left',          
                 indicator=True)) 
print("MERGED CONCESSION DATA FRAME AND CITIES -> LOCALIZATION OF EVERY CONCESSION")
# print(merged.head())

vc = merged['_merge'].value_counts()

print("Dopasowane   :", vc.get('both', 0))
print("Niedopasowane:", vc.get('left_only', 0))
print("Nie­użyte     :", vc.get('right_only', 0))

missing = merged.loc[merged['_merge'] == 'left_only', 'city']

print("Koncesje bez lokalizacji przydzielonego miasta")
print(missing.unique())
percent = vc.get('left_only', 0)/(vc.get('both', 0) + vc.get('left_only', 0))*100

print(f"\n### Dropping {percent:.2f}% of data ###")
merged.dropna(subset=["lon", "lat"], inplace=True)

MERGED CONCESSION DATA FRAME AND CITIES -> LOCALIZATION OF EVERY CONCESSION
Dopasowane   : 394
Niedopasowane: 33
Nie­użyte     : 0
Koncesje bez lokalizacji przydzielonego miasta
['bienkowka' 'nowy  tomysl' 'bielsko - biala' 'libertow' 'dobron'
 'banska nizna' 'tenczynek' 'stare stracze' 'modlniczka' 'tomaszkowo'
 'dziekanow nowy' 'poronin' 'wegrzce' 'uciechow' 'kokoszkowy' 'siedlecin'
 'weglowice' 'proszowki' 'zascianki' 'srock' 'chelm slaski' 'dys'
 'busko-zdroj' 'plock-biala' 'stary bartkow' 'sekocin nowy' 'miedzychow'
 'zbylitowska gora' 'nowiny' 'gilowice' 'stare proboszczewice' 'wolkowo']
Dropping 7.73% of data


# Obliczanie korelacji między pożarami a merged (koncesje & lokalizacja)

In [18]:
df_conc_count = count_concession(merged)

df_corr = (df_conc_count.merge(df_events, on='city', how='inner'))


print(df_conc_count)
pearson = df_corr[['Liczba_koncesji', 'RAZEM Pożar (P)']].corr().iloc[0,1]
print("Pearson =", round(pearson, 3))

                     city  Liczba_koncesji
0    aleksandrow kujawski                1
1               bialogard                1
2               bialystok                6
3         bielsk podlaski                1
4           bielsko-biala                5
..                    ...              ...
176          zdunska wola                2
177              zelazkow                1
178          zielona gora                3
179              zyrardow                1
180                zywiec                2

[181 rows x 2 columns]
Pearson = 0.779
